In [1]:
import numpy as np 
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/lectures/lectures.csv


In [2]:
! pip install simplet5 -q

DEPRECATION: pytorch-lightning 1.5.10 has a non-standard dependency specifier torch>=1.7.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
arviz 0.18.0 requires setuptools>=60.0.0, but you have setuptools 59.5.0 which is incompatible.
conda 23.7.4 requires setuptools>=60.0.0, but you have setuptools 59.5.0 which is incompatible.
kaggle-environments 1.14.3 requires transformers>=4.33.1, but you have transformers 4.16.2 which is incompatible.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you h

In [3]:
df = pd.read_csv("../input/lectures/lectures.csv", encoding='latin-1', usecols=['Lecture', 'Summary'])

In [4]:
df.head()

,Lecture,Summary
0,"Today, we are delving into the captivating wor...","The lecture explores Pre-Socratic philosophy, ..."
1,"Today, we delve into one of the most fascinati...","The lecture explores Plato's Theory of Forms, ..."
2,"Hello everyone, today we are diving into the f...",The lecture explores Aristotle's Virtue Ethics...
3,"Welcome, everyone. Today, we are going to delv...","The lecture explores Stoicism, an ancient phil..."
4,"Welcome, everyone. Today, we are going to dive...",The lecture explores the Eastern philosophy of...


In [5]:
df = df.rename(columns={"Summary":"target_text", "Lecture":"source_text"})
df = df[['source_text', 'target_text']]

In [6]:
df.head()

,source_text,target_text
0,"Today, we are delving into the captivating wor...","The lecture explores Pre-Socratic philosophy, ..."
1,"Today, we delve into one of the most fascinati...","The lecture explores Plato's Theory of Forms, ..."
2,"Hello everyone, today we are diving into the f...",The lecture explores Aristotle's Virtue Ethics...
3,"Welcome, everyone. Today, we are going to delv...","The lecture explores Stoicism, an ancient phil..."
4,"Welcome, everyone. Today, we are going to dive...",The lecture explores the Eastern philosophy of...


In [7]:
df['source_text'] = "summarize: " + df['source_text']

In [8]:
df.head()

,source_text,target_text
0,"summarize: Today, we are delving into the capt...","The lecture explores Pre-Socratic philosophy, ..."
1,"summarize: Today, we delve into one of the mos...","The lecture explores Plato's Theory of Forms, ..."
2,"summarize: Hello everyone, today we are diving...",The lecture explores Aristotle's Virtue Ethics...
3,"summarize: Welcome, everyone. Today, we are go...","The lecture explores Stoicism, an ancient phil..."
4,"summarize: Welcome, everyone. Today, we are go...",The lecture explores the Eastern philosophy of...


In [9]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2)
train_df.shape, test_df.shape

((799, 2), (200, 2))

In [10]:
from simplet5 import SimpleT5

model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-small")

2024-04-24 15:31:28.503925: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-24 15:31:28.504066: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-24 15:31:28.640199: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Downloading:   0%|          | 0.00/2.27k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

In [14]:
model.train(train_df=train_df,
            eval_df=test_df, 
            source_max_token_len=512, 
            target_max_token_len=512, 
            batch_size=10, max_epochs=5, use_gpu=True)

Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [15]:
! ( cd outputs; ls )

simplet5-epoch-0-train-loss-1.9021-val-loss-1.8116
simplet5-epoch-0-train-loss-2.4687-val-loss-1.982
simplet5-epoch-1-train-loss-1.8427-val-loss-1.7788
simplet5-epoch-1-train-loss-2.0774-val-loss-1.8664
simplet5-epoch-2-train-loss-1.7954-val-loss-1.7634
simplet5-epoch-3-train-loss-1.7551-val-loss-1.7482
simplet5-epoch-4-train-loss-1.7155-val-loss-1.7505


In [16]:
model.load_model("t5","outputs/simplet5-epoch-4-train-loss-1.7155-val-loss-1.7505", use_gpu=True)

In [17]:
text_to_summarize="""summarize: Vaccination and safety measures such as wearing face masks are essential when it comes to fighting the Delta Plus coronavirus variant, World Health Organization (WHO) representative to Russia Melita Vujnovic said.

"Vaccination plus masks, because just a vaccine is not enough with 'Delta Plus'. We need to make an effort over a short period of time, otherwise there would be a lockdown," Vujnovic said on the Soloviev Live YouTube show.

She explained that vaccination is essential because it lowers the probability of spreading the virus and lowers the risks of severe disease. However, "additional measures" will probably be required as well, Vujnovic warned.

Earlier in June, the WHO included the Delta variant in its list of coronavirus variants of concern as the strain had become prevalent and has caused a resurgence of COVID-19 cases in some countries, including Russia. India has also reported multiple cases of the Delta Plus strain, which was first discovered in March.
"""
model.predict(text_to_summarize)

['Melita Vujnovic, a World Health Organization representative to Russia, said Vaccination plus masks are essential for combating the Delta Plus coronavirus variant. "We need to make an effort over a short period of time, otherwise there would be a lockdown," Vujnovic said. The WHO included the Delta variant in its list of coronavirus variants earlier in June. India has also reported multiple cases of the Delta Plus strain.']

In [18]:
import pickle
pickle.dump(model,open('outputs/model-small.pkl','wb'))

In [19]:
model_loaded=pickle.load(open('outputs/model-small.pkl','rb'))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [20]:
model_loaded.predict(text_to_summarize)

['Melita Vujnovic, a World Health Organization representative to Russia, said Vaccination plus masks are essential for combating the Delta Plus coronavirus variant. "We need to make an effort over a short period of time, otherwise there would be a lockdown," Vujnovic said. The WHO included the Delta variant in its list of coronavirus variants earlier in June. India has also reported multiple cases of the Delta Plus strain.']

In [1]:
%cd /kaggle/working

/kaggle/working


In [2]:
from IPython.display import FileLink 
FileLink(r'outputs/model-small.pkl')

/kaggle/working/outputs/model-small.pkl